In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 2.0 MB/s eta 0:00:00


In [3]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import ast

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the SentenceTransformer model and move it to the GPU
model = SentenceTransformer('all-MiniLM-L6-v2')
model = model.to(device)

# Step 1: Load input sentences and entity knowledge strings from CSVs
input_df = pd.read_csv('/kaggle/input/business-json/Task_2_data.csv')  # CSV with columns 'sentence' and 'actual_entity'
entity_df = pd.read_csv('/kaggle/input/business-json/Knowledge.csv')  # CSV with columns 'entity_name' and 'entity_description'

# Function to extract entity names
def extract_entity_name(entities_str):
    entities_list = ast.literal_eval(entities_str)
    # Return the first two mentions if they exist, otherwise None
    return (entities_list[0], entities_list[1]) if len(entities_list) > 1 else (entities_list[0], None)

# Apply the function to the 'entities' column and expand into two new columns
input_df[['sentence_1_entity_name', 'sentence_2_entity_name']] = input_df['entity_titles'].apply(extract_entity_name).apply(pd.Series)

# Display the updated DataFrame
print(input_df)

asr_df = pd.read_csv('/kaggle/input/business-json/task_2_asr_out.csv')

/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

     Unnamed: 0  QID                                   question  \
0             0    0  Are these established in the same decade?   
1             1    1  Are these established in the same decade?   
2             2    2  Are these established in the same decade?   
3             3    3  Are these established in the same decade?   
4             4    4  Are these established in the same decade?   
..          ...  ...                                        ...   
495         495  495               Are these greek restaurants?   
496         496  496               Are these greek restaurants?   
497         497  497               Are these greek restaurants?   
498         498  498               Are these greek restaurants?   
499         499  499               Are these greek restaurants?   

                                          sentence_1  \
0      Banco do Brasil has headquarters in Brasília.   
1         Banco do Brasil has origin country Brazil.   
2       Banco do Brasil has

# Full Knowledge

In [4]:
# Step 2: Encode the input sentences and entity knowledge strings using GPU
input1_embeddings = model.encode(asr_df['Sentence_1_transcript'].tolist(), convert_to_tensor=True, device=device)
input2_embeddings = model.encode(asr_df['Sentence_2_transcript'].tolist(), convert_to_tensor=True, device=device)
entity_embeddings = model.encode(entity_df['Knowledge'].tolist(), convert_to_tensor=True, device=device)

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

In [5]:
# Initialize counters for ranking accuracy
top1_correct = 0
top5_correct = 0
top10_correct = 0

top1_correct_s = 0
top5_correct_s = 0
top10_correct_s = 0

r_s = []
p_s = []
f1_s = []
res_data = []

# Step 3: Calculate cosine similarity and evaluate rankings
for i, q_id in enumerate(input_df['QID']):
    # Get the similarity scores for the current sentence with all entities
    similarity_scores1 = cosine_similarity(input1_embeddings[i].unsqueeze(0).cpu().numpy(), entity_embeddings.cpu().numpy()).flatten()
    similarity_scores2 = cosine_similarity(input2_embeddings[i].unsqueeze(0).cpu().numpy(), entity_embeddings.cpu().numpy()).flatten()
    # Get the indices of entities sorted by similarity score (descending order)
    ranked_entity_indices1 = similarity_scores1.argsort()[::-1]
    ranked_entity_indices2 = similarity_scores2.argsort()[::-1]
    # Get the ranked entity names
    ranked_entity_names1 = entity_df['Entity_name'].iloc[ranked_entity_indices1]
    ranked_entity_names2 = entity_df['Entity_name'].iloc[ranked_entity_indices2]

    # Get the actual entity name for the current sentence
    actual_entity_name1 = input_df['sentence_1_entity_name'].iloc[i]
    actual_entity_name2 = input_df['sentence_2_entity_name'].iloc[i]
    
    res_data.append(
    {
        'q_id': q_id,
        'linked_entity_sentnece_1': ranked_entity_names1.iloc[0],
        'actual_entity_sentence_1' : actual_entity_name1,
        'linked_entity_sentnece_2': ranked_entity_names2.iloc[0],
        'actual_entity_sentence_2' : actual_entity_name2,
    }
    )
    count = 0
    # Step 4: Check if the actual entity is within the top 1, 5, and 10
    if actual_entity_name1 == ranked_entity_names1.iloc[0]:
        top1_correct += 1
        count += 1
    if actual_entity_name1 in ranked_entity_names1.iloc[:5].values:
        top5_correct += 1
    if actual_entity_name1 in ranked_entity_names1.iloc[:10].values:
        top10_correct += 1
        
    if actual_entity_name2 == ranked_entity_names2.iloc[0]:
        top1_correct += 1
        count += 1
    if actual_entity_name2 in ranked_entity_names2.iloc[:5].values:
        top5_correct += 1
    if actual_entity_name2 in ranked_entity_names2.iloc[:10].values:
        top10_correct += 1
    if actual_entity_name1 == ranked_entity_names1.iloc[0] and actual_entity_name2 == ranked_entity_names2.iloc[0]:
        top1_correct_s += 1
    if actual_entity_name1 in ranked_entity_names1.iloc[:5].values and actual_entity_name2 in ranked_entity_names2.iloc[:5].values:
        top5_correct_s += 1
    if actual_entity_name1 in ranked_entity_names1.iloc[:10].values and actual_entity_name2 in ranked_entity_names2.iloc[:10].values:
        top10_correct_s += 1
    
    r = p = f1 = count/2
    
    r_s.append(r)
    p_s.append(p)
    f1_s.append(f1)
# Step 5: Calculate and print ranking accuracies
total_sentences = len(input_df)

top1_accuracy = top1_correct / total_sentences * 50
top5_accuracy = top5_correct / total_sentences * 50
top10_accuracy = top10_correct / total_sentences * 50


top1_accuracy_s = top1_correct_s / total_sentences * 100
top5_accuracy_s = top5_correct_s / total_sentences * 100
top10_accuracy_s = top10_correct_s / total_sentences * 100


print(f"Top-1 Accuracy: {top1_accuracy:.2f}%")
print(f"Top-5 Accuracy: {top5_accuracy:.2f}%")
print(f"Top-10 Accuracy: {top10_accuracy:.2f}%")


print(f"Top-1 Set Accuracy: {top1_accuracy_s:.2f}%")
print(f"Top-5 Set Accuracy: {top5_accuracy_s:.2f}%")
print(f"Top-10 Set Accuracy: {top10_accuracy_s:.2f}%")


print(f'average f1 score : {(sum(f1_s)/len(f1_s))}')
print(f'average recall score : {(sum(r_s)/len(r_s))}')
print(f'average precision score : {(sum(p_s)/len(p_s))}')

Top-1 Accuracy: 59.10%
Top-5 Accuracy: 71.70%
Top-10 Accuracy: 77.10%
Top-1 Set Accuracy: 37.00%
Top-5 Set Accuracy: 52.60%
Top-10 Set Accuracy: 59.60%
average f1 score : 0.591
average recall score : 0.591
average precision score : 0.591


In [6]:
ner_df = pd.DataFrame(res_data)

In [7]:
ner_df.to_csv("Task2_asr_entity_linked_CLIP_full_knowledge_results.csv")

# Partial Knowledge

In [8]:
# Step 2: Encode the input sentences and entity knowledge strings using GPU
input1_embeddings = model.encode(asr_df['Sentence_1_transcript'].tolist(), convert_to_tensor=True, device=device)
input2_embeddings = model.encode(asr_df['Sentence_2_transcript'].tolist(), convert_to_tensor=True, device=device)

# Calculate the substring for the first 20% of each knowledge string
entity_df['Knowledge_20'] = entity_df['Knowledge'].apply(lambda x: x[:int(len(x) * 0.2)])

# Encode the first 20% of the knowledge strings
entity_embeddings = model.encode(entity_df['Knowledge_20'].tolist(), convert_to_tensor=True, device=device)


Batches:   0%|          | 0/16 [00:00<?, ?it/s]

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

In [9]:
# Initialize counters for ranking accuracy
top1_correct = 0
top5_correct = 0
top10_correct = 0

top1_correct_s = 0
top5_correct_s = 0
top10_correct_s = 0

r_s = []
p_s = []
f1_s = []
res_data = []

# Step 3: Calculate cosine similarity and evaluate rankings
for i, q_id in enumerate(input_df['QID']):
    # Get the similarity scores for the current sentence with all entities
    similarity_scores1 = cosine_similarity(input1_embeddings[i].unsqueeze(0).cpu().numpy(), entity_embeddings.cpu().numpy()).flatten()
    similarity_scores2 = cosine_similarity(input2_embeddings[i].unsqueeze(0).cpu().numpy(), entity_embeddings.cpu().numpy()).flatten()
    # Get the indices of entities sorted by similarity score (descending order)
    ranked_entity_indices1 = similarity_scores1.argsort()[::-1]
    ranked_entity_indices2 = similarity_scores2.argsort()[::-1]
    # Get the ranked entity names
    ranked_entity_names1 = entity_df['Entity_name'].iloc[ranked_entity_indices1]
    ranked_entity_names2 = entity_df['Entity_name'].iloc[ranked_entity_indices2]

    # Get the actual entity name for the current sentence
    actual_entity_name1 = input_df['sentence_1_entity_name'].iloc[i]
    actual_entity_name2 = input_df['sentence_2_entity_name'].iloc[i]
    
    res_data.append(
    {
        'q_id': q_id,
        'linked_entity_sentnece_1': ranked_entity_names1.iloc[0],
        'actual_entity_sentence_1' : actual_entity_name1,
        'linked_entity_sentnece_2': ranked_entity_names2.iloc[0],
        'actual_entity_sentence_2' : actual_entity_name2,
    }
    )
    count = 0
    # Step 4: Check if the actual entity is within the top 1, 5, and 10
    if actual_entity_name1 == ranked_entity_names1.iloc[0]:
        top1_correct += 1
        count += 1
    if actual_entity_name1 in ranked_entity_names1.iloc[:5].values:
        top5_correct += 1
    if actual_entity_name1 in ranked_entity_names1.iloc[:10].values:
        top10_correct += 1
        
    if actual_entity_name2 == ranked_entity_names2.iloc[0]:
        top1_correct += 1
        count += 1
    if actual_entity_name2 in ranked_entity_names2.iloc[:5].values:
        top5_correct += 1
    if actual_entity_name2 in ranked_entity_names2.iloc[:10].values:
        top10_correct += 1
    if actual_entity_name1 == ranked_entity_names1.iloc[0] and actual_entity_name2 == ranked_entity_names2.iloc[0]:
        top1_correct_s += 1
    if actual_entity_name1 in ranked_entity_names1.iloc[:5].values and actual_entity_name2 in ranked_entity_names2.iloc[:5].values:
        top5_correct_s += 1
    if actual_entity_name1 in ranked_entity_names1.iloc[:10].values and actual_entity_name2 in ranked_entity_names2.iloc[:10].values:
        top10_correct_s += 1
    
    r = p = f1 = count/2
    
    r_s.append(r)
    p_s.append(p)
    f1_s.append(f1)
# Step 5: Calculate and print ranking accuracies
total_sentences = len(input_df)

top1_accuracy = top1_correct / total_sentences * 50
top5_accuracy = top5_correct / total_sentences * 50
top10_accuracy = top10_correct / total_sentences * 50


top1_accuracy_s = top1_correct_s / total_sentences * 100
top5_accuracy_s = top5_correct_s / total_sentences * 100
top10_accuracy_s = top10_correct_s / total_sentences * 100


print(f"Top-1 Accuracy: {top1_accuracy:.2f}%")
print(f"Top-5 Accuracy: {top5_accuracy:.2f}%")
print(f"Top-10 Accuracy: {top10_accuracy:.2f}%")


print(f"Top-1 Set Accuracy: {top1_accuracy_s:.2f}%")
print(f"Top-5 Set Accuracy: {top5_accuracy_s:.2f}%")
print(f"Top-10 Set Accuracy: {top10_accuracy_s:.2f}%")


print(f'average f1 score : {(sum(f1_s)/len(f1_s))}')
print(f'average recall score : {(sum(r_s)/len(r_s))}')
print(f'average precision score : {(sum(p_s)/len(p_s))}')

Top-1 Accuracy: 55.20%
Top-5 Accuracy: 67.30%
Top-10 Accuracy: 71.00%
Top-1 Set Accuracy: 32.80%
Top-5 Set Accuracy: 45.60%
Top-10 Set Accuracy: 50.20%
average f1 score : 0.552
average recall score : 0.552
average precision score : 0.552


In [10]:
ner_df = pd.DataFrame(res_data)

In [11]:
ner_df.to_csv("Task2_asr_entity_linked_CLIP_partial_knowledge_results.csv")

# Labels Only

In [12]:
# Step 2: Encode the input sentences and entity knowledge strings using GPU
input1_embeddings = model.encode(asr_df['Sentence_1_transcript'].tolist(), convert_to_tensor=True, device=device)
input2_embeddings = model.encode(asr_df['Sentence_2_transcript'].tolist(), convert_to_tensor=True, device=device)
entity_embeddings = model.encode(entity_df['Entity_name'].tolist(), convert_to_tensor=True, device=device)

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

In [13]:
# Initialize counters for ranking accuracy
top1_correct = 0
top5_correct = 0
top10_correct = 0

top1_correct_s = 0
top5_correct_s = 0
top10_correct_s = 0

r_s = []
p_s = []
f1_s = []
res_data = []

# Step 3: Calculate cosine similarity and evaluate rankings
for i, q_id in enumerate(input_df['QID']):
    # Get the similarity scores for the current sentence with all entities
    similarity_scores1 = cosine_similarity(input1_embeddings[i].unsqueeze(0).cpu().numpy(), entity_embeddings.cpu().numpy()).flatten()
    similarity_scores2 = cosine_similarity(input2_embeddings[i].unsqueeze(0).cpu().numpy(), entity_embeddings.cpu().numpy()).flatten()
    # Get the indices of entities sorted by similarity score (descending order)
    ranked_entity_indices1 = similarity_scores1.argsort()[::-1]
    ranked_entity_indices2 = similarity_scores2.argsort()[::-1]
    # Get the ranked entity names
    ranked_entity_names1 = entity_df['Entity_name'].iloc[ranked_entity_indices1]
    ranked_entity_names2 = entity_df['Entity_name'].iloc[ranked_entity_indices2]

    # Get the actual entity name for the current sentence
    actual_entity_name1 = input_df['sentence_1_entity_name'].iloc[i]
    actual_entity_name2 = input_df['sentence_2_entity_name'].iloc[i]
    
    res_data.append(
    {
        'q_id': q_id,
        'linked_entity_sentnece_1': ranked_entity_names1.iloc[0],
        'actual_entity_sentence_1' : actual_entity_name1,
        'linked_entity_sentnece_2': ranked_entity_names2.iloc[0],
        'actual_entity_sentence_2' : actual_entity_name2,
    }
    )
    count = 0
    # Step 4: Check if the actual entity is within the top 1, 5, and 10
    if actual_entity_name1 == ranked_entity_names1.iloc[0]:
        top1_correct += 1
        count += 1
    if actual_entity_name1 in ranked_entity_names1.iloc[:5].values:
        top5_correct += 1
    if actual_entity_name1 in ranked_entity_names1.iloc[:10].values:
        top10_correct += 1
        
    if actual_entity_name2 == ranked_entity_names2.iloc[0]:
        top1_correct += 1
        count += 1
    if actual_entity_name2 in ranked_entity_names2.iloc[:5].values:
        top5_correct += 1
    if actual_entity_name2 in ranked_entity_names2.iloc[:10].values:
        top10_correct += 1
    if actual_entity_name1 == ranked_entity_names1.iloc[0] and actual_entity_name2 == ranked_entity_names2.iloc[0]:
        top1_correct_s += 1
    if actual_entity_name1 in ranked_entity_names1.iloc[:5].values and actual_entity_name2 in ranked_entity_names2.iloc[:5].values:
        top5_correct_s += 1
    if actual_entity_name1 in ranked_entity_names1.iloc[:10].values and actual_entity_name2 in ranked_entity_names2.iloc[:10].values:
        top10_correct_s += 1
    
    r = p = f1 = count/2
    
    r_s.append(r)
    p_s.append(p)
    f1_s.append(f1)
# Step 5: Calculate and print ranking accuracies
total_sentences = len(input_df)

top1_accuracy = top1_correct / total_sentences * 50
top5_accuracy = top5_correct / total_sentences * 50
top10_accuracy = top10_correct / total_sentences * 50


top1_accuracy_s = top1_correct_s / total_sentences * 100
top5_accuracy_s = top5_correct_s / total_sentences * 100
top10_accuracy_s = top10_correct_s / total_sentences * 100


print(f"Top-1 Accuracy: {top1_accuracy:.2f}%")
print(f"Top-5 Accuracy: {top5_accuracy:.2f}%")
print(f"Top-10 Accuracy: {top10_accuracy:.2f}%")


print(f"Top-1 Set Accuracy: {top1_accuracy_s:.2f}%")
print(f"Top-5 Set Accuracy: {top5_accuracy_s:.2f}%")
print(f"Top-10 Set Accuracy: {top10_accuracy_s:.2f}%")


print(f'average f1 score : {(sum(f1_s)/len(f1_s))}')
print(f'average recall score : {(sum(r_s)/len(r_s))}')
print(f'average precision score : {(sum(p_s)/len(p_s))}')

Top-1 Accuracy: 55.60%
Top-5 Accuracy: 66.10%
Top-10 Accuracy: 71.60%
Top-1 Set Accuracy: 32.00%
Top-5 Set Accuracy: 43.80%
Top-10 Set Accuracy: 51.60%
average f1 score : 0.556
average recall score : 0.556
average precision score : 0.556


In [14]:
ner_df = pd.DataFrame(res_data)

In [15]:
ner_df.to_csv("Task2_asr_entity_linked_CLIP_label_results.csv")